# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    source_id_text = [[source_vocab_to_int[word] for word in sentence.split()] for sentence in source_text.split('\n')]
    
    target_id_text = [[target_vocab_to_int[word] for word in sentence.split()] + [target_vocab_to_int['<EOS>']] for sentence in target_text.split('\n')]
    
    
    return source_id_text, target_id_text


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0


/home/aash/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.
  from ipykernel import kernelapp as app


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input_int_text = tf.placeholder(tf.int32, [None, None], name ='input')
    targets_int_text = tf.placeholder(tf.int32, [None, None], name = 'targets')
    learning_rate = tf.placeholder(tf.float32, name = 'learning_rate')
    keep_probability = tf.placeholder(tf.float32, name = 'keep_prob')
    target_seq_length = tf.placeholder(tf.int32, (None,), name = 'target_sequence_length')
    max_target_seq_length = tf.reduce_max(target_seq_length)
    source_seq_length = tf.placeholder(tf.int32,(None,), name = 'source_sequence_length')
    
    
    return input_int_text, targets_int_text, learning_rate, keep_probability, target_seq_length,max_target_seq_length, source_seq_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    processed_target = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)
    return processed_target
    

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length,source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    
    enc_embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    def make_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return enc_cell
    
    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    enc_cell = tf.contrib.rnn.DropoutWrapper(enc_cell, output_keep_prob=keep_prob)
                      
    enc_output, enc_state = tf.nn.dynamic_rnn(enc_cell, enc_embed_input, sequence_length=source_sequence_length, dtype=tf.float32)
                   
    return enc_output, enc_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    
    helper = tf.contrib.seq2seq.TrainingHelper(dec_embed_input, target_sequence_length, time_major=False)
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, helper, encoder_state, output_layer)
    
    final_outputs, _ = tf.contrib.seq2seq.dynamic_decode(decoder, impute_finished=True,
                                                         maximum_iterations=max_summary_length)
    

    
    return final_outputs



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')
    
    helper =  tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, start_tokens, end_of_sequence_id)
    
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, helper, encoder_state, output_layer)
    
    finaloutputs, _  = tf.contrib.seq2seq.dynamic_decode(decoder,impute_finished=True,
                                                         maximum_iterations=max_target_sequence_length)
    return finaloutputs



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    start_of_sequence_id = target_vocab_to_int['<GO>']
    end_of_sequence_id = target_vocab_to_int['<EOS>']

    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
      
    #decoder cell
    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return dec_cell

    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
       
    output_layer = Dense(target_vocab_size, kernel_initializer =
                         tf.truncated_normal_initializer(mean=0.0, stddev=0.1 ))
          
    with tf.variable_scope('decode'):       
       
       
        train_basic_dec_output = decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length,        
                                                      max_target_sequence_length, output_layer, keep_prob)
       
       
       
    with tf.variable_scope('decode', reuse=True):                              
        
        inf_basic_dec_output = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                                                       end_of_sequence_id, max_target_sequence_length, target_vocab_size,
                                                       output_layer, batch_size, keep_prob)
       
       
       
       
    return train_basic_dec_output, inf_basic_dec_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Apply embedding to the input data for the encoder.
- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Apply embedding to the target data for the decoder.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [14]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    inp_encode, enc_state = encoding_layer(input_data, rnn_size, num_layers, keep_prob,  source_sequence_length, 
                   source_vocab_size, enc_embedding_size)
    
    processed_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)    

    training_logits, inference_logits = decoding_layer(processed_input, enc_state, target_sequence_length, 
                                       max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, 
                                       target_vocab_size, batch_size, keep_prob, dec_embedding_size)
    
    
    return training_logits, inference_logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [15]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 64
# RNN Size
rnn_size = 32
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 64
decoding_embedding_size = 64
# Learning Rate
learning_rate = 0.005
# Dropout Keep Probability
keep_probability = 0.8
display_step = 4

### Build the Graph
Build the graph using the neural network you implemented.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    4/2154 - Train Accuracy: 0.2172, Validation Accuracy: 0.2841, Loss: 5.4041
Epoch   0 Batch    8/2154 - Train Accuracy: 0.1850, Validation Accuracy: 0.2841, Loss: 4.7235
Epoch   0 Batch   12/2154 - Train Accuracy: 0.2445, Validation Accuracy: 0.2841, Loss: 4.0614
Epoch   0 Batch   16/2154 - Train Accuracy: 0.2352, Validation Accuracy: 0.2997, Loss: 3.7437
Epoch   0 Batch   20/2154 - Train Accuracy: 0.2797, Validation Accuracy: 0.3189, Loss: 3.5451
Epoch   0 Batch   24/2154 - Train Accuracy: 0.2790, Validation Accuracy: 0.3161, Loss: 3.4912
Epoch   0 Batch   28/2154 - Train Accuracy: 0.3021, Validation Accuracy: 0.3189, Loss: 3.4300
Epoch   0 Batch   32/2154 - Train Accuracy: 0.2467, Validation Accuracy: 0.3182, Loss: 3.5174
Epoch   0 Batch   36/2154 - Train Accuracy: 0.2836, Validation Accuracy: 0.3182, Loss: 3.2928
Epoch   0 Batch   40/2154 - Train Accuracy: 0.2262, Validation Accuracy: 0.3189, Loss: 3.4147
Epoch   0 Batch   44/2154 - Train Accuracy: 0.2383, Validati

Epoch   0 Batch  356/2154 - Train Accuracy: 0.3766, Validation Accuracy: 0.4268, Loss: 1.4643
Epoch   0 Batch  360/2154 - Train Accuracy: 0.3709, Validation Accuracy: 0.4439, Loss: 1.4282
Epoch   0 Batch  364/2154 - Train Accuracy: 0.4062, Validation Accuracy: 0.4276, Loss: 1.3796
Epoch   0 Batch  368/2154 - Train Accuracy: 0.3898, Validation Accuracy: 0.4020, Loss: 1.4342
Epoch   0 Batch  372/2154 - Train Accuracy: 0.3413, Validation Accuracy: 0.4162, Loss: 1.4298
Epoch   0 Batch  376/2154 - Train Accuracy: 0.3734, Validation Accuracy: 0.4062, Loss: 1.3461
Epoch   0 Batch  380/2154 - Train Accuracy: 0.3461, Validation Accuracy: 0.4254, Loss: 1.3372
Epoch   0 Batch  384/2154 - Train Accuracy: 0.4112, Validation Accuracy: 0.4084, Loss: 1.2194
Epoch   0 Batch  388/2154 - Train Accuracy: 0.4375, Validation Accuracy: 0.3878, Loss: 1.2726
Epoch   0 Batch  392/2154 - Train Accuracy: 0.3773, Validation Accuracy: 0.3935, Loss: 1.3163
Epoch   0 Batch  396/2154 - Train Accuracy: 0.3561, Validati

Epoch   0 Batch  708/2154 - Train Accuracy: 0.4885, Validation Accuracy: 0.4993, Loss: 0.9323
Epoch   0 Batch  712/2154 - Train Accuracy: 0.4836, Validation Accuracy: 0.5078, Loss: 0.8438
Epoch   0 Batch  716/2154 - Train Accuracy: 0.4918, Validation Accuracy: 0.4972, Loss: 0.8087
Epoch   0 Batch  720/2154 - Train Accuracy: 0.5058, Validation Accuracy: 0.5128, Loss: 0.8613
Epoch   0 Batch  724/2154 - Train Accuracy: 0.5016, Validation Accuracy: 0.5014, Loss: 0.8711
Epoch   0 Batch  728/2154 - Train Accuracy: 0.5102, Validation Accuracy: 0.5213, Loss: 0.8348
Epoch   0 Batch  732/2154 - Train Accuracy: 0.4926, Validation Accuracy: 0.5142, Loss: 0.8461
Epoch   0 Batch  736/2154 - Train Accuracy: 0.5109, Validation Accuracy: 0.4979, Loss: 0.7466
Epoch   0 Batch  740/2154 - Train Accuracy: 0.5406, Validation Accuracy: 0.5071, Loss: 0.7918
Epoch   0 Batch  744/2154 - Train Accuracy: 0.4898, Validation Accuracy: 0.5000, Loss: 0.8012
Epoch   0 Batch  748/2154 - Train Accuracy: 0.5335, Validati

Epoch   0 Batch 1060/2154 - Train Accuracy: 0.5703, Validation Accuracy: 0.5639, Loss: 0.6374
Epoch   0 Batch 1064/2154 - Train Accuracy: 0.5633, Validation Accuracy: 0.5618, Loss: 0.6273
Epoch   0 Batch 1068/2154 - Train Accuracy: 0.5567, Validation Accuracy: 0.5888, Loss: 0.6658
Epoch   0 Batch 1072/2154 - Train Accuracy: 0.5687, Validation Accuracy: 0.5938, Loss: 0.6225
Epoch   0 Batch 1076/2154 - Train Accuracy: 0.5872, Validation Accuracy: 0.5916, Loss: 0.6848
Epoch   0 Batch 1080/2154 - Train Accuracy: 0.5672, Validation Accuracy: 0.6108, Loss: 0.6231
Epoch   0 Batch 1084/2154 - Train Accuracy: 0.5852, Validation Accuracy: 0.5902, Loss: 0.6390
Epoch   0 Batch 1088/2154 - Train Accuracy: 0.5750, Validation Accuracy: 0.5938, Loss: 0.6283
Epoch   0 Batch 1092/2154 - Train Accuracy: 0.6164, Validation Accuracy: 0.6058, Loss: 0.6592
Epoch   0 Batch 1096/2154 - Train Accuracy: 0.5773, Validation Accuracy: 0.6080, Loss: 0.6618
Epoch   0 Batch 1100/2154 - Train Accuracy: 0.5273, Validati

Epoch   0 Batch 1412/2154 - Train Accuracy: 0.6480, Validation Accuracy: 0.6165, Loss: 0.5475
Epoch   0 Batch 1416/2154 - Train Accuracy: 0.6241, Validation Accuracy: 0.6143, Loss: 0.5610
Epoch   0 Batch 1420/2154 - Train Accuracy: 0.6382, Validation Accuracy: 0.6037, Loss: 0.5552
Epoch   0 Batch 1424/2154 - Train Accuracy: 0.6570, Validation Accuracy: 0.5973, Loss: 0.5310
Epoch   0 Batch 1428/2154 - Train Accuracy: 0.6275, Validation Accuracy: 0.6165, Loss: 0.5630
Epoch   0 Batch 1432/2154 - Train Accuracy: 0.6406, Validation Accuracy: 0.5866, Loss: 0.5115
Epoch   0 Batch 1436/2154 - Train Accuracy: 0.6176, Validation Accuracy: 0.6193, Loss: 0.5263
Epoch   0 Batch 1440/2154 - Train Accuracy: 0.6094, Validation Accuracy: 0.6023, Loss: 0.5488
Epoch   0 Batch 1444/2154 - Train Accuracy: 0.6406, Validation Accuracy: 0.6087, Loss: 0.5323
Epoch   0 Batch 1448/2154 - Train Accuracy: 0.6503, Validation Accuracy: 0.6122, Loss: 0.5022
Epoch   0 Batch 1452/2154 - Train Accuracy: 0.6258, Validati

...

Epoch   9 Batch  700/2154 - Train Accuracy: 0.9572, Validation Accuracy: 0.9787, Loss: 0.0277
Epoch   9 Batch  704/2154 - Train Accuracy: 0.9441, Validation Accuracy: 0.9467, Loss: 0.0363
Epoch   9 Batch  708/2154 - Train Accuracy: 0.9457, Validation Accuracy: 0.9496, Loss: 0.0515
Epoch   9 Batch  712/2154 - Train Accuracy: 0.9375, Validation Accuracy: 0.9503, Loss: 0.0408
Epoch   9 Batch  716/2154 - Train Accuracy: 0.8929, Validation Accuracy: 0.9482, Loss: 0.0389
Epoch   9 Batch  720/2154 - Train Accuracy: 0.9630, Validation Accuracy: 0.9538, Loss: 0.0324
Epoch   9 Batch  724/2154 - Train Accuracy: 0.9613, Validation Accuracy: 0.9474, Loss: 0.0403
Epoch   9 Batch  728/2154 - Train Accuracy: 0.8688, Validation Accuracy: 0.9304, Loss: 0.0490
Epoch   9 Batch  732/2154 - Train Accuracy: 0.9770, Validation Accuracy: 0.9190, Loss: 0.0257
Epoch   9 Batch  736/2154 - Train Accuracy: 0.9492, Validation Accuracy: 0.9254, Loss: 0.0341
Epoch   9 Batch  740/2154 - Train Accuracy: 0.9555, Validati

Epoch   9 Batch 1052/2154 - Train Accuracy: 0.9605, Validation Accuracy: 0.9205, Loss: 0.0497
Epoch   9 Batch 1056/2154 - Train Accuracy: 0.9572, Validation Accuracy: 0.9496, Loss: 0.0300
Epoch   9 Batch 1060/2154 - Train Accuracy: 0.9164, Validation Accuracy: 0.9474, Loss: 0.0387
Epoch   9 Batch 1064/2154 - Train Accuracy: 0.9336, Validation Accuracy: 0.9247, Loss: 0.0413
Epoch   9 Batch 1068/2154 - Train Accuracy: 0.9474, Validation Accuracy: 0.9389, Loss: 0.0317
Epoch   9 Batch 1072/2154 - Train Accuracy: 0.9273, Validation Accuracy: 0.9467, Loss: 0.0398
Epoch   9 Batch 1076/2154 - Train Accuracy: 0.9243, Validation Accuracy: 0.9659, Loss: 0.0342
Epoch   9 Batch 1080/2154 - Train Accuracy: 0.9172, Validation Accuracy: 0.9730, Loss: 0.0525
Epoch   9 Batch 1084/2154 - Train Accuracy: 0.9258, Validation Accuracy: 0.9680, Loss: 0.0348
Epoch   9 Batch 1088/2154 - Train Accuracy: 0.9422, Validation Accuracy: 0.9609, Loss: 0.0248
Epoch   9 Batch 1092/2154 - Train Accuracy: 0.9461, Validati

Epoch   9 Batch 1404/2154 - Train Accuracy: 0.9515, Validation Accuracy: 0.9446, Loss: 0.0337
Epoch   9 Batch 1408/2154 - Train Accuracy: 0.9273, Validation Accuracy: 0.9396, Loss: 0.0484
Epoch   9 Batch 1412/2154 - Train Accuracy: 0.9424, Validation Accuracy: 0.9467, Loss: 0.0300
Epoch   9 Batch 1416/2154 - Train Accuracy: 0.9470, Validation Accuracy: 0.9531, Loss: 0.0337
Epoch   9 Batch 1420/2154 - Train Accuracy: 0.9309, Validation Accuracy: 0.9411, Loss: 0.0403
Epoch   9 Batch 1424/2154 - Train Accuracy: 0.8992, Validation Accuracy: 0.9254, Loss: 0.0623
Epoch   9 Batch 1428/2154 - Train Accuracy: 0.9219, Validation Accuracy: 0.9411, Loss: 0.0473
Epoch   9 Batch 1432/2154 - Train Accuracy: 0.9313, Validation Accuracy: 0.9489, Loss: 0.0278
Epoch   9 Batch 1436/2154 - Train Accuracy: 0.9597, Validation Accuracy: 0.9389, Loss: 0.0315
Epoch   9 Batch 1440/2154 - Train Accuracy: 0.9352, Validation Accuracy: 0.9588, Loss: 0.0316
Epoch   9 Batch 1444/2154 - Train Accuracy: 0.9289, Validati

Epoch   9 Batch 1756/2154 - Train Accuracy: 0.9609, Validation Accuracy: 0.9411, Loss: 0.0188
Epoch   9 Batch 1760/2154 - Train Accuracy: 0.9416, Validation Accuracy: 0.9425, Loss: 0.0323
Epoch   9 Batch 1764/2154 - Train Accuracy: 0.9586, Validation Accuracy: 0.9467, Loss: 0.0396
Epoch   9 Batch 1768/2154 - Train Accuracy: 0.8824, Validation Accuracy: 0.9403, Loss: 0.0534
Epoch   9 Batch 1772/2154 - Train Accuracy: 0.9883, Validation Accuracy: 0.9510, Loss: 0.0183
Epoch   9 Batch 1776/2154 - Train Accuracy: 0.9268, Validation Accuracy: 0.9553, Loss: 0.0500
Epoch   9 Batch 1780/2154 - Train Accuracy: 0.9469, Validation Accuracy: 0.9652, Loss: 0.0281
Epoch   9 Batch 1784/2154 - Train Accuracy: 0.9745, Validation Accuracy: 0.9531, Loss: 0.0219
Epoch   9 Batch 1788/2154 - Train Accuracy: 0.9398, Validation Accuracy: 0.9638, Loss: 0.0338
Epoch   9 Batch 1792/2154 - Train Accuracy: 0.9398, Validation Accuracy: 0.9709, Loss: 0.0332
Epoch   9 Batch 1796/2154 - Train Accuracy: 0.9554, Validati

Epoch   9 Batch 2108/2154 - Train Accuracy: 0.9546, Validation Accuracy: 0.9645, Loss: 0.0224
Epoch   9 Batch 2112/2154 - Train Accuracy: 0.9457, Validation Accuracy: 0.9574, Loss: 0.0311
Epoch   9 Batch 2116/2154 - Train Accuracy: 0.9539, Validation Accuracy: 0.9602, Loss: 0.0384
Epoch   9 Batch 2120/2154 - Train Accuracy: 0.9293, Validation Accuracy: 0.9439, Loss: 0.0463
Epoch   9 Batch 2124/2154 - Train Accuracy: 0.9523, Validation Accuracy: 0.9510, Loss: 0.0523
Epoch   9 Batch 2128/2154 - Train Accuracy: 0.9334, Validation Accuracy: 0.9489, Loss: 0.0344
Epoch   9 Batch 2132/2154 - Train Accuracy: 0.9406, Validation Accuracy: 0.9695, Loss: 0.0411
Epoch   9 Batch 2136/2154 - Train Accuracy: 0.9625, Validation Accuracy: 0.9688, Loss: 0.0510
Epoch   9 Batch 2140/2154 - Train Accuracy: 0.9353, Validation Accuracy: 0.9702, Loss: 0.0291
Epoch   9 Batch 2144/2154 - Train Accuracy: 0.9301, Validation Accuracy: 0.9361, Loss: 0.0422
Epoch   9 Batch 2148/2154 - Train Accuracy: 0.9430, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [22]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    sentence = sentence.lower()
    words = []
    for word in sentence.split(' '):
        if not word:
            continue 
        else:
            try:
                words.append(vocab_to_int[word])
            except KeyError: 
                words.append(vocab_to_int['<UNK>'])                
        
    return words


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [30]:
translate_sentence = 'she is going to school .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [8, 67, 133, 79, 53, 170]
  English Words: ['she', 'is', 'going', 'to', 'school', '.']

Prediction
  Word Ids:      [48, 166, 211, 313, 357, 67, 1]
  French Words: elle va à août prochain . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.